# Unofficial ACI Guide

## Python 3 - Get All Tenant Endpoint Groups and their Endpoints Example

This is a simple Python example demonstrating how to get tenants, endpoint groups, and endpoints.  
No Cisco ACI-related toolkits, SDKs, or bindings were harmed in the creation of this example.

In [1]:
import requests
import json
import pandas as pd
import numpy as np

In [2]:
# Variables to construct our URLs.
# Configure username, password, and controller in config.py.
# See aci-auth.ipynb notebook for examples on getting user/pass
# inline, from config files, and from input prompts.

from config import controller, username, password
base_url = "https://" + str(controller) + "/api/"
auth_bit = "aaaLogin.json"

auth_url = base_url + auth_bit

In [3]:
# JSON auth data 
auth_data = {
  "aaaUser":{
    "attributes":{
      "name":username,
      "pwd":password
    }
  }
}

## Construct the Request

In [4]:
# We use verify=False to allow self-signed certs. 
# The disable_warnings() line prevents warning messages.
requests.packages.urllib3.disable_warnings() 
s = requests.session()
s.post(auth_url, json=auth_data, verify=False)

<Response [200]>

### Get all EPGs

In [5]:
c_string = "?rsp-subtree=children"
epg_path="node/class/fvAEPg.json"
epg_url = base_url + epg_path + c_string
epg_url

'https://10.18.188.101/api/node/class/fvAEPg.json?rsp-subtree=children'

In [11]:
# Set the value of "epg_response" to the output of a GET request to the EPG URL.
# Then take that output and turn it into a Python object (a dictionary).
epg_response = s.get(epg_url, verify=False)
epg_json = epg_response.json()

In [7]:
print(json.dumps(epg_json, indent=4, sort_keys=True))
# The response is a JSON dictionary with two keys: imdata and totalCount.
# The value for imdata is a list of objects, while the value of totalCount is a count of those objects. 
# Thd totalCount can be used to cross-check any data handling so that you don't 'lose' objects 
# during any filtering/processing/handling.

{
    "imdata": [
        {
            "fvAEPg": {
                "attributes": {
                    "annotation": "",
                    "childAction": "",
                    "configIssues": "",
                    "configSt": "applied",
                    "descr": "Digitial Signing Server",
                    "dn": "uni/tn-EPIC/ap-EPIC/epg-Digital_SS",
                    "exceptionTag": "",
                    "extMngdBy": "",
                    "floodOnEncap": "disabled",
                    "fwdCtrl": "",
                    "hasMcastSource": "no",
                    "isAttrBasedEPg": "no",
                    "isSharedSrvMsiteEPg": "no",
                    "lcOwn": "local",
                    "matchT": "AtleastOne",
                    "modTs": "2018-11-17T15:11:23.786-05:00",
                    "monPolDn": "uni/tn-common/monepg-default",
                    "name": "Digital_SS",
                    "nameAlias": "",
                    "pcEnfPref": "unenforced",
     

In [13]:
# Read the imdata key, and assign the value (a list of objects) to epg_objects_list.
epg_objects_list = epg_json['imdata']
epg_objects_list

[{'fvAEPg': {'attributes': {'annotation': '',
    'childAction': '',
    'configIssues': '',
    'configSt': 'applied',
    'descr': 'Digitial Signing Server',
    'dn': 'uni/tn-EPIC/ap-EPIC/epg-Digital_SS',
    'exceptionTag': '',
    'extMngdBy': '',
    'floodOnEncap': 'disabled',
    'fwdCtrl': '',
    'hasMcastSource': 'no',
    'isAttrBasedEPg': 'no',
    'isSharedSrvMsiteEPg': 'no',
    'lcOwn': 'local',
    'matchT': 'AtleastOne',
    'modTs': '2018-11-17T15:11:23.786-05:00',
    'monPolDn': 'uni/tn-common/monepg-default',
    'name': 'Digital_SS',
    'nameAlias': '',
    'pcEnfPref': 'unenforced',
    'pcTag': '32784',
    'prefGrMemb': 'exclude',
    'prio': 'unspecified',
    'scope': '2818048',
    'shutdown': 'no',
    'status': '',
    'triggerSt': 'triggerable',
    'txId': '15564440312192464251',
    'uid': '15374'},
   'children': [{'fvRsProv': {'attributes': {'annotation': '',
       'childAction': '',
       'ctrctUpd': 'ctrct',
       'extMngdBy': '',
       'force

In [9]:
#Set up some empty lists to capture the data we need. 
tenant_list = []
ap_list = []
epg_list = []
endpoint_list = []
ip_list = []
mac_list = []
encap_list = []

In [10]:
# Now we have a list of objects, let's iterate over each object and do the following:
#  - Grab the Distriguished name (DN)
#  - Split the DN string into a list using the forward slash as a separator
#  - Ignore uni, which is element 0.
#  - Then, grab the tenant from element 1
#  - the Application profile (AP) is element 2
#  - Skip to the last element (-1) to grab the endpoint group.
#  - In the aci-get-global-endpoints example, we describe why we read from the end.
#  - But basically, the last element is always endpoint, but the fourth (3) element is not always
#    the last element.

for epg_object in epg_objects_list:
    #print(object)
    dn = epg_object['fvAEPg']['attributes']['dn']
    #print(child)
    #print(dn)
    split_dn = dn.split("/")
    tenant = split_dn[1]
    ap = split_dn[2]
    epg = split_dn[-1]
    
    # Get the child objects, which are values under the key "children".
    # We also set null value for endpoint, ip address, mac, and encapsulation.
    # Not every EPG will have and endpoint, so we want to keep a placeholder.
    
    children_list = epg_object['fvAEPg']['children']
    endpoint = None
    ip = None
    mac = None
    encap = None
    for child in children_list:
        #print(child.items())
        
        # For each child object, we look for anything in the class "fvCEp", which is an endpoint.
        # If it is, we overwrite the null values for endpoint name, ip, mac, and encapsulation.
        # If it isn't fvCEp, we skip it and just keep the placeholders above.
        # Uncomment the print statements if you want to see the data.
        if "fvCEp" in child:
            endpoint = child['fvCEp']['attributes']['name']
            ip = child['fvCEp']['attributes']['ip']
            mac = child['fvCEp']['attributes']['mac']
            encap = child['fvCEp']['attributes']['encap']
    #print(tenant, ap, epg, endpoint, ip, mac, encap)
    # Append our extracted values into each of the lists we created above.
    tenant_list.append(tenant)
    ap_list.append(ap)
    epg_list.append(epg)
    endpoint_list.append(endpoint)
    ip_list.append(ip)
    mac_list.append(mac)
    encap_list.append(encap)

In [11]:
# Zip the lists into a zip object. This effectively turns then into tuples of 
# tenant, application profile, EPG, endpoint, etc.
tn_ep_list = zip(tenant_list, ap_list, epg_list, endpoint_list, ip_list, mac_list, encap_list)

In [12]:
# Take the zip of tuples, convert to list, and stuff them into a Pandas dataframe.
# Also, convert "None" to numpy NaN, though Pandas deals with "None" just fine for groupby objects. 
df_input = list(tn_ep_list)
df = pd.DataFrame(df_input, columns=("Tenant","AP","EPG","Endpoint","IP","MAC Addy","Encap"))
df.fillna(value=pd.np.nan, inplace=True)

In [13]:
# Sort dataframe by Endpoints and then print the first 20 rows
df.sort_values(by=['Endpoint'], ascending=False).head(20)

# Or sort dataframe by Endpoints and then print it all.
#df.sort_values(by=['Endpoint'], ascending=False)

# Or you can just print the dataframe:
# df

,Tenant,AP,EPG,Endpoint,IP,MAC Addy,Encap
18,tn-COAST,ap-app1,epg-epg1b,00:50:56:93:6C:74,101.1.1.102,00:50:56:93:6C:74,vlan-1102
0,tn-EPIC,ap-EPIC,epg-Digital_SS,NaN,NaN,NaN,NaN
1,tn-EPIC,ap-EPIC,epg-CareLinkWeb,NaN,NaN,NaN,NaN
2,tn-EPIC,ap-EPIC,epg-CommProxy,NaN,NaN,NaN,NaN
3,tn-EPIC,ap-EPIC,epg-MyChart,NaN,NaN,NaN,NaN
4,tn-EPIC,ap-EPIC,epg-IVR_Server,NaN,NaN,NaN,NaN
5,tn-EPIC,ap-EPIC,epg-WarpDrive,NaN,NaN,NaN,NaN
6,tn-EPIC,ap-EPIC,epg-WebBlob,NaN,NaN,NaN,NaN
7,tn-EPIC2,ap-EPIC,epg-CacheDB,NaN,NaN,NaN,NaN
8,tn-EPIC4,ap-EPIC,epg-Clarity,NaN,NaN,NaN,NaN


In [14]:
# Report total number of EPGs and EPs per Tenant using groupby object.
grouped_df = df.groupby(['Tenant']).agg({'EPG':'count','Endpoint':'count'})
grouped_df

,EPG,Endpoint
Tenant,,
tn-COAST,6,1
tn-EPIC,28,0
tn-EPIC2,5,0
tn-EPIC3,5,0
tn-EPIC4,5,0
tn-JW_Tenant,1,0
tn-J_Tenant,3,0
tn-andrew,28,0
tn-infra,2,0


In [15]:
# We can drop all rows with any empty fields.
df = df.dropna()

In [16]:
df

,Tenant,AP,EPG,Endpoint,IP,MAC Addy,Encap
18,tn-COAST,ap-app1,epg-epg1b,00:50:56:93:6C:74,101.1.1.102,00:50:56:93:6C:74,vlan-1102
